## Keyphrase grouping

The grouping algorithm is given a number of lists of keyphrases, each list describes one document. Additionaly it may be given those documents (it is up to you to decide). The task is to output partitions of keyphrases that should be merged together and the reference keyphrase they be merged into. E.g. ["nuclear energy", "wind energy", "energy"] -> "energy" or ["wind energy", "hydropower", "solar energy"] -> "renewable energy sources" (the second example is more difficult to achieve). The algorithm may use any publicly available data or tools, e.g. word embeddings, language models, onthologies or dictionaries.

In [106]:
from nltk.corpus import wordnet as wn

In [108]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mmastalerczyk/nltk_data...


True

In [110]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mmastalerczyk/nltk_data...


True

In [196]:
def get_synsets(word):
    pl_synsets = wn.synsets(word, lang='pol')
    if len(pl_synsets):
        return set(pl_synsets)
    return set(wn.synsets(word)) # sometimes in pol sysnet we can't get proper names, like opel

In [307]:
def get_common_hypernyms(synsets):
    synsets_list = list(synsets)
    if len(synsets_list) == 1:
        return set(synsets_list[0].hypernyms()).union(synsets)
    hypernyms = set()
    for i in range(1, len(synsets_list)):
        current_hypernyms = set(synsets_list[i-1].lowest_common_hypernyms(synsets_list[i]))
        hypernyms.update(current_hypernyms)
    return list(hypernyms)

In [308]:
def get_hypernyms_counter(synsets_list):
    hypernyms_list = []
    for i in range(1, len(synsets_list)):
        current_hypernyms = set(synsets_list[i-1].lowest_common_hypernyms(synsets_list[i]))
        hypernyms_list.extend(current_hypernyms)
    from collections import Counter
    counted_hypernyms = Counter(hypernyms_list)
    return counted_hypernyms

In [339]:
def get_one_word_summary(counter):
    counter = dict(counter)
    max_value = max(counter.values())
    synsets_list = []
    for key, value in counter.items():
        if value == max_value:
            synsets_list.append(key)
    if len(synsets_list) == 1:
        return synsets_list[0]
    hypernyms_cunter = get_hypernyms_counter(synsets_list)
    return get_one_word_summary(hypernyms_cunter)

In [366]:
def process_keyword_list(keyword_list):
    hypernyms_list = []
    for keyword in keyword_list:
        base_synsets = get_synsets(keyword)
        keyword_hypernyms = get_common_hypernyms(base_synsets)
        hypernyms_list.extend(keyword_hypernyms)
    print(hypernyms_list)
    hypernyms_counter = get_hypernyms_counter(hypernyms_list)
    summary_keyword = get_one_word_summary(hypernyms_counter)
    if summary_keyword.lemmas(lang='pol'):
        return summary_keyword.lemmas(lang='pol')
    return summary_keyword

In [341]:
process_keyword_list(['pies', 'kot', 'krowa'])

[Lemma('organism.n.01.egzemplarz'),
 Lemma('organism.n.01.jednostka'),
 Lemma('organism.n.01.organizm'),
 Lemma('organism.n.01.osobnik')]

In [342]:
process_keyword_list(['Tesla', 'Opel', 'Toyota'])

Synset('entity.n.01')

In [343]:
process_keyword_list(['krzesło', 'stół'])

[Lemma('furniture.n.01.mebel')]

In [348]:
process_keyword_list(['pizza', 'makaron', 'mięso', 'ryż'])

[Lemma('dish.n.02.potrawa')]

In [355]:
process_keyword_list(['żyrafa', 'słoń', 'zebra'])

[Lemma('placental.n.01.ssak_łożyskowy'), Lemma('placental.n.01.łożyskowiec')]

In [364]:
process_keyword_list(['książka', 'artykół', 'czasopismo', 'biografia'])

[Lemma('publication.n.01.pozycja'),
 Lemma('publication.n.01.tytuł'),
 Lemma('publication.n.01.wydawnictwo')]

In [369]:
process_keyword_list(['nóż', 'łyżka', 'widelec'])

[Synset('knife.n.01'), Synset('container.n.01'), Synset('spoon.n.01'), Synset('cutlery.n.02'), Synset('fork.n.01')]


[Lemma('cutlery.n.02.sztuciec')]

In [371]:
process_keyword_list(['skarpeta', 'spodnie', 'koszula'])

[Synset('entity.n.01'), Synset('garment.n.01'), Synset('trouser.n.01'), Synset('garment.n.01'), Synset('shirt.n.01')]


[Lemma('garment.n.01.część_garderoby'),
 Lemma('garment.n.01.element_garderoby'),
 Lemma('garment.n.01.element_ubioru'),
 Lemma('garment.n.01.sztuka_odzieży')]

In [373]:
process_keyword_list(['plecy', 'głowa', 'brzuch', 'ręka'])

[Synset('entity.n.01'), Synset('entity.n.01'), Synset('body_part.n.01'), Synset('abdomen.n.01'), Synset('entity.n.01')]


Synset('entity.n.01')

##  Keyphrase Extraction

The extracting algorithm is given a set of documents and should output a set of lists of phrases (one list of phrases per document) that best describe (summarize) the document content. Those phrases should be present in the document. Lemmatization of the keyphrases is optional. Additionaly the algorith should rank those phrases, meaning that the phrase more important for the document (better describing its content) should be given higher score. E.g. in the article about Orlen-Lotos merger, the companies names would probably score higher than names of the CEOs, while the names of people quoted in the article could not be extracted as keywords at all. The algorithm may or may not process one document at a time. The algorithm may use any publicly available data or tools, e.g. word embeddings, language models, onthologies or dictionaries

In [ ]:
# https://medium.com/mlearning-ai/10-popular-keyword-extraction-algorithms-in-natural-language-processing-8975ada5750c

In [58]:
from multi_rake import Rake

def extract_keyphrases(text):
#     rake = Rake(language_code="pl") don't need pl, interfere automatically
    rake = Rake()
    keywords = rake.apply(text)
    print(keywords)

In [59]:
rok_polskiej_reprezentacji = """
    Drużyna narodowa kończy rok 2022 – tak jak 2021 – bez selekcjonera. Nie musiało tak być, 
    ale prezes Cezary Kulesza nie miał wyjścia. W przypadku Czesława Michniewicza wiele mogło 
    pójść w zapomnienie, gdyby do piłki nożnej nie została tak chętnie zaproszona polityka. 
    Nie było nikogo rozsądnego, kto zasugerowałby premierowi Mateuszowi Morawieckiemu, że proponowanie 
    kilkudziesięciu mln zł piłkarzom jest złym pomysłem w czasach kryzysu, inflacji i drożyzny. 
    A jak było wcześniej? Oto rok piłkarskiej reprezentacji Polski.
"""

In [60]:
extract_keyphrases(rok_polskiej_reprezentacji)

[('prezes cezary kulesza', 9.0), ('przypadku czesława michniewicza', 9.0), ('chętnie zaproszona polityka', 9.0), ('miał wyjścia', 4.0), ('mogło pójść', 4.0), ('piłki nożnej', 4.0), ('nikogo rozsądnego', 4.0), ('złym pomysłem', 4.0), ('czasach kryzysu', 4.0), ('selekcjonera', 1.0), ('musiało', 1.0), ('zapomnienie', 1.0), ('została', 1.0), ('inflacji', 1.0), ('drożyzny', 1.0), ('wcześniej', 1.0), ('2021', 0)]


In [61]:
rok_polskiej_reprezentacji_long = """
    Drużyna narodowa kończy rok 2022 – tak jak 2021 – bez selekcjonera. Nie musiało tak być, 
    ale prezes Cezary Kulesza nie miał wyjścia. W przypadku Czesława Michniewicza wiele mogło 
    pójść w zapomnienie, gdyby do piłki nożnej nie została tak chętnie zaproszona polityka. 
    Nie było nikogo rozsądnego, kto zasugerowałby premierowi Mateuszowi Morawieckiemu, że proponowanie 
    kilkudziesięciu mln zł piłkarzom jest złym pomysłem w czasach kryzysu, inflacji i drożyzny. 
    A jak było wcześniej? Oto rok piłkarskiej reprezentacji Polski.
    Dwanaście miesięcy temu po jesiennej zmianie władz w związku, kiedy Cezary Kulesza zastąpił 
    Zbigniewa Bońka, Portugalczyk Paulo Sousa niespodziewanie podjął decyzję o ucieczce z Polski, 
    przyjmując propozycję brazylijskiego Flamengo Rio de Janeiro (zapłacił naszej federacji za 
    zerwanie umowy), w którym popracował ledwie pół roku.
    Teraz Czesław Michniewicz nie dostał nowego kontraktu od prezesa Kuleszy. Szef federacji drugi 
    rok z rzędu na przełomie grudnia i stycznia musi szukać trenera najważniejszej drużyny w kraju. 
    Czas nie nagli – Polska dopiero 24 marca rozpoczyna eliminacje Euro 2024, a grupowi rywale są 
    wyjątkowo sprzyjający i nie wymagają działań nagłych: Czechy, Albania, Wyspy Owcze oraz Mołdawia.
    Do turnieju, który w 2024 r. zostanie rozegrany w Niemczech, zakwalifikują się dwie najlepsze 
    reprezentacje. Początek kwalifikacji będzie jednak istotny – 24 marca Biało-Czerwoni zagrają 
    w Czechach, trzy dni później na PGE Narodowym w Warszawie podejmiemy Albanię. O tym, 
    co będzie – mniej więcej wiadomo. Następcą Michniewicza ma zostać obcokrajowiec. Poniżej 
    przypominamy dwanaście miesięcy reprezentacji Polski.
"""

In [62]:
extract_keyphrases(rok_polskiej_reprezentacji_long)

[('prezes cezary kulesza', 9.0), ('chętnie zaproszona polityka', 9.0), ('dwanaście miesięcy temu', 9.0), ('jesiennej zmianie władz', 9.0), ('zapłacił naszej federacji', 9.0), ('popracował ledwie pół', 9.0), ('dostał nowego kontraktu', 9.0), ('wymagają działań nagłych', 9.0), ('trzy dni później', 9.0), ('warszawie podejmiemy albanię', 9.0), ('przypadku czesława michniewicza', 8.5), ('następcą michniewicza', 4.5), ('miał wyjścia', 4.0), ('mogło pójść', 4.0), ('piłki nożnej', 4.0), ('nikogo rozsądnego', 4.0), ('złym pomysłem', 4.0), ('czasach kryzysu', 4.0), ('zerwanie umowy', 4.0), ('czesław michniewicz', 4.0), ('prezesa kuleszy', 4.0), ('przełomie grudnia', 4.0), ('grupowi rywale', 4.0), ('wyjątkowo sprzyjający', 4.0), ('wyspy owcze', 4.0), ('zostanie rozegrany', 4.0), ('najlepsze reprezentacje', 4.0), ('początek kwalifikacji', 4.0), ('pge narodowym', 4.0), ('zostać obcokrajowiec', 4.0), ('selekcjonera', 1.0), ('musiało', 1.0), ('zapomnienie', 1.0), ('została', 1.0), ('inflacji', 1.0), 

In [63]:
benedykt_xvi_nie_zyje = """
Emerytowany papież Benedykt XVI, kard. Joseph Ratzinger, nie żyje. Miał 95 lat. Informację o jego śmierci 
przekazał dyrektor biura prasowego Watykanu Matteo Bruni.
Bruni przekazał również, że Benedykt XVI zmarł o godzinie 9.34. Jak dodał, ciało emerytowanego papieża 
zostanie wystawione w Bazylice Świętego Piotra, aby "wierni mogli go pożegnać".
W środę pojawiły się informacje o poważnym stanie emerytowanego papieża Benedykta XVI. —Jest bardzo 
chory — wyjawił Franciszek, który odwiedził swojego poprzednika i apelował o modlitwę za niego.
Emerytowany papież miał prawdopodobnie problemy z oddychaniem, które nasiliły się przed Bożym Narodzeniem 
— podawała włoska agencja Ansa. Amerykańska gazeta katolicka "National Catholic Register" informowała, 
powołując się na źródło w Watykanie, że Benedykt XVI mógł cierpieć na niewydolność nerek.
Benedykt XVI po abdykacji w 2013 r. pozostał w Watykanie i mieszkał w klauzurowym zakonie Matter Ecclesiae. 
Od czasu do czasu w sieci pojawiały się zdjęcia Josepha Ratzingera. Jedno z ostatnich udostępnione zostało 
w listopadzie.
Joseph Ratzinger urodził się 16 kwietnia 1927 r. w Marktl am Inn w Bawarii. Miał dwójkę rodzeństwa – brata 
Georga, który także był księdzem i zmarł w 2020 r. oraz siostrę Marię, która była jego gosposią 
aż do śmierci w 1991 r. Rodzina przyszłego papieża kilkukrotnie musiała się przeprowadzać. Według doniesień 
medialnych powodem była antynazistowska postawa ojca. Jako nastolatek Joseph wraz z rodziną zamieszkali 
w Traunstein na granicy z Austrią.
Ratzinger w 1939 r. wstąpił do niższego seminarium duchownego, które jednak ze względu na działania wojenne 
musiał opuścić. Przyszły papież w wieku 14 lat został zapisany do Hitlerjugend, co było obowiązkowe w tamtych 
czasach. Gdy skończył 16 lat, został wcielony do wojska, gdzie służył m.in. jako wartownik i telefonista. 
Przyszły papież otrzymał przydział do koszar Wermachtu i służył na różnych posterunkach wokół Traunstein. 
Pod koniec wojny zdezerterował.
W latach 1946-1951 studiował filozofię i teologię na Uniwersytecie w Monachium oraz w szkole wyższej we Freisingu. 
Wraz z bratem Georgem przyjął święcenia kapłańskie 29 czerwca 1951 r. W 1953 r. ks. Ratzinger uzyskał tytuł 
doktora teologii. Cztery lata później uzyskał tytuł profesora uniwersyteckiego. Wykładał dogmatykę i teologię 
w Wyższej Szkole Filozoficznej i Teologicznej we Freisingu, w Bonn, Münsterze i Tybindze. Od 1969 r. był 
profesorem teologii dogmatycznej i historii dogmatyki na Uniwersytecie w Ratyzbonie, gdzie ostatecznie został 
dziekanem i prorektorem.
Uczestniczył we wszystkich czterech sesjach Soboru Watykańskiego II jako główny doradca teologiczny arcybiskupa 
Kolonii, kardynała Josefa Fringsa. W 1977 r. papież Paweł VI powołał ks. Ratzingera na arcybiskupa Monachium 
i Freisingu, a miesiąc później kreował go kardynałem.
W 1981 r. kard. Ratzinger został mianowany przez papieża Jana Pawła II prefektem Kongregacji Nauki Wiary. 
Został także przewodniczącym Komisji Biblijnej i Papieskiej Międzynarodowej Komisji Teologicznej. W latach 
1986-92 pracował jako przewodniczący komisji zajmującej się opracowaniem nowego Katechizmu Kościoła Katolickiego.
W 1998 r. został mianowany prodziekanem, a w 2002 r. dziekanem Kolegium Kardynałów. Pracował też w Sekretariacie 
Stanu, Kongregacji ds. Kościołów Wschodnich, Kongregacji ds. Kultu Bożego i Dyscypliny Sakramentów, Kongregacji 
ds. Biskupów i wielu innych. 8 kwietnia 2005 r., kardynał Ratzinger przewodniczył mszy świętej podczas pogrzebu 
Jana Pawła II na placu św. Piotra.
19 kwietnia Joseph Ratzinger został wybrany 265. papieżem i przybrał imię Benedykt XVI. Po swoim wyborze zwrócił 
się do wiernych zgromadzonych na Placu Świętego Piotra słowami: "Umiłowani bracia i siostry. Po wielkim papieżu 
Janie Pawle II kardynałowie wybrali mnie – prostego, skromnego pracownika winnicy Pana. Pocieszam się faktem, 
że Pan potrafi się posługiwać niedoskonałymi narzędziami i działać przy ich pomocy, a przede wszystkim 
zawierzam się waszym modlitwom. W radości Pana zmartwychwstałego, ufni w Jego stałą pomoc, idziemy przed siebie. 
Pan nam pomoże, a Maryja, Jego Najświętsza Matka, stoi u naszego boku. Dziękuję".
Benedykt XVI odbył w sumie 24 zagraniczne podróże apostolskie. W dniach 25-28 maja 2006 r. odwiedził Polskę. 
Była to jego jedyna wizyta w naszym kraju i druga po Niemczech zagraniczna pielgrzymka papieża. Wizyta papieża 
z Niemiec w byłym nazistowskim obozie koncentracyjnym Auschwitz była symboliczna, szczególnie że przyszły papież 
w swojej młodości był zmuszony do służby w Hitlerjugend.
W lutym 2013 r. Benedykt XVI ogłosił, że z końcem miesiąca rezygnuje z funkcji biskupa Rzymu i odchodzi 
na emeryturę. Wywołało to powszechne zaskoczenie na całym świecie. 28 lutego o godz. 20 Benedykt XVI przestał 
być papieżem. Ostatni raz takie wydarzenia miało miejsce w XIII w. Po rezygnacji prosił, żeby nazywać go po 
prostu "ojciec Benedykt".
Ustępujący papież przebywał jakiś czas w letniej rezydencji w Castel Gandolfo, aby potem przenieść się do 
ogrodów watykańskich i klauzurowego zakonu Matter Ecclesiae.
Benedykt XVI od czasu ustąpienia unikał publicznych wystąpień. Pojawiał się jednak czasem na zaproszenie 
papieża Franciszka. I tak wziął udział m.in. w odsłonięciu pomnika Michała Archanioła w Watykanie (2013 r.), 
kanonizacji papieży Jana XXIII i Jana Pawła II (2014 r.), otwarciu drzwi świętych w Roku Miłosierdzia (2015 r.). 
Sporadycznie w sieci ukazywały się też zdjęcia emerytowanego papieża.
Swoją ostatnią podróż zagraniczną Benedykt XVI odbył w 2020 r., już po wybuchu epidemii koronawirusa. Papież 
emeryt na pięć dni poleciał do Niemiec, gdzie odwiedził swojego umierającego starszego brata Georga oraz groby 
rodziców i siostry. Wydarzenie to wzbudziło wielkie zainteresowanie mediów. Świat obiegły wówczas zdjęcia wyraźnie 
schorowanego Benedykta XVI poruszającego się na wózku inwalidzkim.
"""

In [64]:
extract_keyphrases(benedykt_xvi_nie_zyje)

[('amerykańska gazeta katolicka', 9.0), ('national catholic register', 9.0), ('ostatnich udostępnione zostało', 9.0), ('marktl am inn', 9.0), ('doniesień medialnych powodem', 9.0), ('antynazistowska postawa ojca', 9.0), ('niższego seminarium duchownego', 9.0), ('koniec wojny zdezerterował', 9.0), ('profesorem teologii dogmatycznej', 9.0), ('kardynała josefa fringsa', 9.0), ('miesiąc później kreował', 9.0), ('przewodniczącym komisji biblijnej', 9.0), ('przewodniczący komisji zajmującej', 9.0), ('swoim wyborze zwrócił', 9.0), ('skromnego pracownika winnicy', 9.0), ('posługiwać niedoskonałymi narzędziami', 9.0), ('końcem miesiąca rezygnuje', 9.0), ('funkcji biskupa rzymu', 9.0), ('wydarzenia miało miejsce', 9.0), ('jana pawła ii', 9.0), ('otwarciu drzwi świętych', 9.0), ('wybuchu epidemii koronawirusa', 9.0), ('pięć dni poleciał', 9.0), ('zaproszenie papieża franciszka', 8.666666666666666), ('zdjęcia emerytowanego papieża', 8.666666666666666), ('odwiedził swojego poprzednika', 8.5), ('wzi

In [65]:
para_zyje_na_bali = """
Wracamy z kolejnym odcinkiem cyklu "Nowi emigranci", w którym piszemy o Polakach mieszkających za granicą. 
Tym razem porozmawialiśmy z Marianną i Piotrem, którzy przeprowadzili się na indonezyjską wyspę Bali.
Jak pokazuje opublikowane w 2021 r. badanie "Migracje Zarobkowe Polaków", aż 16 proc. pracujących mieszkańców 
naszego kraju w ciągu najbliższych 12 miesięcy rozważa emigrację zarobkową. To dwa razy więcej niż przed pandemią, 
na co wpływ ma spadek dochodów wielu osób, a także wzrost inflacji. Najczęściej o wyjeździe myślą osoby 
w wieku 24-35 lat, dlatego postanowiliśmy ruszyć na Noizz z cyklem "Nowi emigranci". Będziemy w nim rozmawiać 
z młodymi ludźmi, którzy już zdecydowali się na ten ruch i mogą się podzielić swoimi doświadczeniami.
Nasi kolejni rozmówcy przeprowadzili się na piękną indonezyjską wyspę Bali, gdzie według serwisu Polonia.org 
obywatele naszego kraju nie tylko wypoczywają, ale coraz częściej także mieszkają na stałe. Marianna i 
Piotr Ciereszyńscy opowiedzieli nam, jak wygląda tam ich życie.
Czemu akurat Bali?
Bali zaczęło się od mojej samotnej podróży po Azji w 2011 r. W planie był tylko miesiąc w Indiach, ale takie 
samotne podróże nigdy nie są samotne, jeśli jest się otwartym na przygody. Skończyło się na kilku miesiącach 
dookoła Azji, w tym nieplanowanym wypadzie na Bali. Poczułam jakąś niesamowitą więź z tym miejscem, jakbym 
już kiedyś tam mieszkała. Wiedziałam, że kiedyś wrócę tam na dłużej. To trochę magiczna historia, bo wówczas 
wspięłam się na wulkan Batur i pomyślałam życzenie, by poznać człowieka, który będzie na tyle szalony, żeby 
przeprowadzić się ze mną na Bali. Tak się stało. Praktycznie od razu po powrocie z Azji poznałam Piotrka.
Jak długo tam jesteście i czym się zajmujecie?
W sumie jesteśmy tu już siedem lat z przerwami. Wracaliśmy w międzyczasie do Polski, próbowaliśmy też pomieszkać 
w innych krajach, ale Bali działa na nas jak magnes. Czujemy, że to jest nasze miejsce, przynajmniej na teraz. 
Kocham Bali i opowiadam o życiu na tej wyspie na swoim blogu Balistka, pomagam również rodzinom, które chcą 
się tu przeprowadzić, pozałatwiać wszystkie formalności i znaleźć odpowiedzi na różne pytania. Blog jest 
zajęciem dodatkowym, natomiast na co dzień zajmuję się szeroko pojętą architekturą i zarządzaniem, czyli 
projektowaniem wnętrz, przeróbkami, budowami i ich nadzorem, poszukiwaniem nieruchomości pod inwestycje, 
zarządzaniem zespołami technicznymi oraz spinaniem wszystkich procesów z tym związanych.
ała moja praca dzieje się na Bali, Piotrka natomiast zdalnie. Jest animatorem, ilustratorem i autorem 
podcastu "Motion Kiosk". Pracuje głównie z krajami anglojęzycznymi, przede wszystkim z Australią, co też 
jest bardzo wygodne ze względu na nieduże różnice czasowe z Bali. W Polsce natomiast stara się promować motion 
design i jest zapraszany na różne wydarzenia branżowe. Ostatnio był jednym z jurorów w konkursie "Polish Graphic 
Design", więc kontakt z krajem ciągle mamy.
"""

In [66]:
extract_keyphrases(para_zyje_na_bali)

[('kolejnym odcinkiem cyklu', 9.0), ('migracje zarobkowe polaków', 9.0), ('wyjeździe myślą osoby', 9.0), ('podzielić swoimi doświadczeniami', 9.0), ('piotr ciereszyńscy opowiedzieli', 9.0), ('mojej samotnej podróży', 9.0), ('trochę magiczna historia', 9.0), ('swoim blogu balistka', 9.0), ('szeroko pojętą architekturą', 9.0), ('wygodne ze względu', 9.0), ('nieduże różnice czasowe', 9.0), ('polish graphic design', 9.0), ('krajem ciągle mamy', 9.0), ('promować motion design', 8.5), ('różne wydarzenia branżowe', 8.5), ('kolejni rozmówcy przeprowadzili', 8.0), ('zarządzaniem zespołami technicznymi', 8.0), ('indonezyjską wyspę bali', 7.666666666666667), ('azji poznałam piotrka', 7.0), ('różne pytania', 4.5), ('motion kiosk', 4.5), ('nowi emigranci', 4.0), ('polakach mieszkających', 4.0), ('razem porozmawialiśmy', 4.0), ('pokazuje opublikowane', 4.0), ('pracujących mieszkańców', 4.0), ('spadek dochodów', 4.0), ('wzrost inflacji', 4.0), ('wieku 24-35', 4.0), ('postanowiliśmy ruszyć', 4.0), ('m

In [67]:
potezne_wybuchy_w_kijowie = """
Trwa 311. dzień inwazji Rosji na Ukrainę. Około południa w całej Ukrainie słychać było syreny alarmowe. Rosjanie 
zaatakowali pociskami rakietowymi Kramatorsk i Chmielnicki, odgłosy eksplozji słychać w Mikołajewie, w Zaporożu 
i Chersoniu. Do serii potężnych eksplozji doszło w Kijowie. W stolicy zginęła jedna osoba, a liczba osób 
hospitalizowanych wynosi powyżej 20. Do zniszczeń doszło w rejonach Peczerskim i Hołosijiwskim. Zapraszamy do 
śledzenia relacji na żywo.
"""

In [68]:
extract_keyphrases(potezne_wybuchy_w_kijowie)

[('dzień inwazji rosji', 9.0), ('całej ukrainie słychać', 9.0), ('odgłosy eksplozji słychać', 9.0), ('syreny alarmowe', 4.0), ('stolicy zginęła', 4.0), ('zniszczeń doszło', 4.0), ('rejonach peczerskim', 4.0), ('śledzenia relacji', 4.0), ('trwa 311', 1.0), ('ukrainę', 1.0), ('południa', 1.0), ('chmielnicki', 1.0), ('mikołajewie', 1.0), ('zaporożu', 1.0), ('chersoniu', 1.0), ('kijowie', 1.0), ('osoba', 1.0), ('hołosijiwskim', 1.0), ('zapraszamy', 1.0), ('żywo', 1.0)]


In [69]:
runal_pomnik_ronaldo = """
Wydarzyło się to, co jeszcze kilka miesięcy temu brzmiałoby jak fikcja; piękna opowieść, której zabraknie szczęśliwego 
zakończenia. Cristiano Ronaldo po ponad 20 latach kariery pełnej sukcesów opuszcza Europę, by występować w saudyjskim 
Al-Nassr. Klub ogłosił podpisanie kontraktu z Portugalczykiem, nazywając go "największym sportowcem w historii". 
Wydaje się jednak, że to miano nie powinno opisywać człowieka, który w taki sposób żegna się z kibicami uważającymi 
go za legendę. Ronaldo zaprzeczył własnym słowom sprzed kilku lat, a kibice mogą wytknąć mu hipokryzję.
"""

In [70]:
extract_keyphrases(runal_pomnik_ronaldo)

[('miesięcy temu brzmiałoby', 9.0), ('zabraknie szczęśliwego zakończenia', 9.0), ('piękna opowieść', 4.0), ('cristiano ronaldo', 4.0), ('saudyjskim al-nassr', 4.0), ('największym sportowcem', 4.0), ('opisywać człowieka', 4.0), ('kibicami uważającymi', 4.0), ('wydarzyło', 1.0), ('fikcja', 1.0), ('występować', 1.0), ('portugalczykiem', 1.0), ('nazywając', 1.0), ('historii', 1.0), ('wydaje', 1.0), ('miano', 1.0), ('żegna', 1.0), ('legendę', 1.0), ('kibice', 1.0), ('wytknąć', 1.0), ('hipokryzję', 1.0)]


In [71]:
runal_pomnik_ronaldo_long = """
Wydarzyło się to, co jeszcze kilka miesięcy temu brzmiałoby jak fikcja; piękna opowieść, której zabraknie szczęśliwego 
zakończenia. Cristiano Ronaldo po ponad 20 latach kariery pełnej sukcesów opuszcza Europę, by występować w saudyjskim 
Al-Nassr. Klub ogłosił podpisanie kontraktu z Portugalczykiem, nazywając go "największym sportowcem w historii". 
Wydaje się jednak, że to miano nie powinno opisywać człowieka, który w taki sposób żegna się z kibicami uważającymi 
go za legendę. Ronaldo zaprzeczył własnym słowom sprzed kilku lat, a kibice mogą wytknąć mu hipokryzję.
Portugalczyk bardzo ceni sobie dbałość o własny wizerunek i to, by zostać zapamiętanym jako jeden z najlepszych 
(a może i najlepszy?) piłkarz w historii futbolu. Choć jego kariera nie była pozbawiona kontrowersji i momentów 
słabości, to ostatecznie kibice jego dawnych klubów czuli do niego ogromną sympatię. W Manchesterze cieszył się 
uwielbieniem, później tak samo było w Madrycie. Pobyt w Turynie również przyniósł wiele momentów wartych zapamiętania, 
a powrót do Anglii miał być dopełnieniem niesamowitej historii wypełnionej sukcesami.
Tym bardziej może dziwić więc to, że na koniec stało się inaczej. Wydawało się, że nic nie jest w stanie zburzyć 
pomnika Cristiano Ronaldo w oczach kibiców Czerwonych Diabłów, ale styl rozstania z klubem wpłynął na odbiór 
Portugalczyka. Przez lata był na Old Trafford idolem, a po głośnym wywiadzie u Piersa Morgana wielu fanów domagało 
się natychmiastowego rozwiązania jego kontraktu.
Widowiskowy kres legendy na Old Trafford nie wydarzył się w jednym momencie, prowadziło do niego wiele mniejszych, 
ale również głośnych wydarzeń. Już latem 2022 r. Ronaldo opuścił przedsezonowe zgrupowanie United, a nieoficjalnie 
mówiło się, że powodem jest chęć znalezienia sobie nowego klubu, który zagra w Lidze Mistrzów. Potem pięciokrotny 
zdobywca Złotej Piłki głośno wyrażał niezadowolenie z powodu spędzania kolejnych meczów na ławce rezerwowych.
Do samego końca pobytu w Anglii napastnik uważał się za piłkarza, który jest w stanie odmienić losy każdego 
spotkania, za kluczowy element układanki Erika ten Haga. Ronaldo nie dawał w tym sezonie na boisku argumentów, 
by dalej być w centrum ofensywy United, a gdy Holender przestał na niego stawiać, zawodnik w głośnym wywiadzie 
dał upust swojej frustracji.
37-latek nie potrafił pogodzić się z tym, że z wiekiem nie jest już w stanie samodzielnie błyszczeć w ofensywie 
Czerwonych Diabłów, a gdy mógł sprawdzić się w roli bardzo cennego, niemal wymarzonego rezerwowego, zdecydował 
się wylać żale w rozmowie w programie Piersa Morgana. Zaprzyjaźniony dziennikarz wydawał się rozumieć jego słowa 
o rozczarowaniu stagnacją Manchesteru United, ale sam klub i jego kibice byli znacznie mniej wyrozumiali wobec 
niesławnego wywiadu.
Nie ulega wątpliwości, że to nie słabsza forma, lecz właśnie słowa wypowiedziane w telewizyjnym występie były 
momentem radykalnej zmiany w postrzeganiu dawnego idola przez kibiców z Old Trafford i powodem jego odejścia z 
klubu. W tym momencie runął jego pomnik w oczach fanów, przelała się czara goryczy. Gdyby nie pamiętny występ w 
telewizji, Ronaldo najpewniej wciąż byłby piłkarzem United. A nawet gdyby w zimowym oknie transferowym zdecydował 
się zmienić zespół na taki, w którym mógłby grać więcej, jego postrzeganie byłoby zupełnie inne.
Czerwone Diabły poinformowały o rozwiązaniu umowy Portugalczyka 22 listopada, na dwa dni przed pierwszym meczem 
jego reprezentacji na mundialu. Po raz pierwszy w karierze Ronaldo przystępował do mistrzostw świata jako bezrobotny, 
a miliony kibiców z całego świata mogły zastanawiać się, jaki będzie kolejny krok jednego z najlepszych piłkarzy 
w dwóch pierwszych dekadach XXI w. 37-latek nie zalicza się już przecież do młodzieńców i można było spodziewać się, 
że kolejny klub może być jego ostatnim. Media łączyły go z Borussią Dortmund, pojawiały się również plotki o 
sentymentalnym powrocie do Lizbony.
"""

In [73]:
extract_keyphrases(runal_pomnik_ronaldo_long)

[('miesięcy temu brzmiałoby', 9.0), ('zabraknie szczęśliwego zakończenia', 9.0), ('dawnych klubów czuli', 9.0), ('widowiskowy kres legendy', 9.0), ('samego końca pobytu', 9.0), ('centrum ofensywy united', 9.0), ('stanie samodzielnie błyszczeć', 9.0), ('ofensywie czerwonych diabłów', 9.0), ('niemal wymarzonego rezerwowego', 9.0), ('zaprzyjaźniony dziennikarz wydawał', 9.0), ('momentem radykalnej zmiany', 9.0), ('postrzeganiu dawnego idola', 9.0), ('postrzeganie byłoby zupełnie', 9.0), ('czerwone diabły poinformowały', 9.0), ('momentów wartych zapamiętania', 8.5), ('anglii napastnik uważał', 8.5), ('kolejny krok jednego', 8.5), ('programie piersa morgana', 8.0), ('karierze ronaldo przystępował', 8.0), ('old trafford idolem', 7.666666666666667), ('piersa morgana', 5.0), ('old trafford', 4.666666666666667), ('momentów słabości', 4.5), ('anglii miał', 4.5), ('piękna opowieść', 4.0), ('cristiano ronaldo', 4.0), ('saudyjskim al-nassr', 4.0), ('największym sportowcem', 4.0), ('opisywać człowie

In [74]:
Warszawa_odmawia_przyjęcia_prymatu_UE = """
Od dłuższego czasu narastają nieporozumienia w kontaktach Polski z UE. Jarosław Kaczyński może wierzyć, że działania 
rewizyjne prowadzone w stosunku do rządu jego partii są skutkiem niedokończonych reform państwa od 2015 r. W 
rzeczywistości to wynik aktywności instytucji powołanych do przeciwdziałania autorytaryzmowi w kraju.
"""

In [75]:
extract_keyphrases(Warszawa_odmawia_przyjęcia_prymatu_UE)

[('działania rewizyjne prowadzone', 9.0), ('kontaktach polski', 4.0), ('jarosław kaczyński', 4.0), ('przeciwdziałania autorytaryzmowi', 4.0), ('wierzyć', 1.0), ('stosunku', 1.0), ('rządu', 1.0), ('partii', 1.0), ('2015 r', 1.0), ('rzeczywistości', 1.0), ('kraju', 1.0)]


In [76]:
Warszawa_odmawia_przyjęcia_prymatu_UE_long = """
Od dłuższego czasu narastają nieporozumienia w kontaktach Polski z UE. Jarosław Kaczyński może wierzyć, że działania 
rewizyjne prowadzone w stosunku do rządu jego partii są skutkiem niedokończonych reform państwa od 2015 r. W 
rzeczywistości to wynik aktywności instytucji powołanych do przeciwdziałania autorytaryzmowi w kraju.
W ostatnich miesiącach utarło się przekonanie, że wojna zmieniła politykę Europy, przesuwając jej środek ciężkości 
na Wschód. Żaden kraj nie wydawał się odzwierciedlać tego przesunięcia bardziej niż Polska, która szybko stała się, 
według słów przewodniczącej Komisji Europejskiej Ursuli von Der Leyen, "świecącym przykładem" europejskiej 
solidarności. Kraj przyjął ponad cztery miliony ukraińskich uchodźców i planuje podwoić swoje długoterminowe wydatki 
na obronę do 5 proc. PKB – jest to zdecydowanie największy wzrost w Europie i wyraźny kontrast z głównymi siłami UE, 
Niemcami i Francją.
W uznaniu ogromnej roli, jaką Polska odegrała w wojennej mobilizacji Europy, Komisja Europejska przedstawiła długo 
odkładany plan naprawczy po pandemii COVID–19 dla Polski, znany jako Krajowy Plan Odbudowy Polski (KPO), zobowiązując 
się do przekazania prawie 36 mld euro w postaci dotacji i subwencji pod warunkiem osiągnięcia przez Polskę pewnych 
"kamieni milowych" dotyczących reform służących przywróceniu praworządności.
Rzeczywistość uderza w PiS
Polscy przywódcy szybko zaczęli wyrażać nadzieję na otrzymanie pierwszych płatności do końca 2022 roku. Ten gest 
dobrej woli ze strony KE był postrzegany jako znak ocieplenia więzi między Warszawą a Brukselą i potencjalne 
zakończenie gorzkiego sporu trwającego od 2015 roku między Prawem i Sprawiedliwością a Unią Europejską o kwestie 
praworządności. Przez chwilę to, co łączyło Brukselę i Warszawę, wydawało się silniejsze od tego, co je dzieliło.
Miesiąc miodowy skończył się jednak, zanim się naprawdę zaczął. Negocjacje między Brukselą a Warszawą zamarły, bo 
Polska nie złożyła jeszcze nawet wymaganego wniosku o fundusze. W październiku PiS odwołał swojego proeuropejskiego 
ministra ds. europejskich i zastąpił go znacznie bardziej radykalnym Szymonem Szynkowskim. "Jeśli Bruksela nadal 
wstrzymuje finansowanie odbudowy po pandemii, to niech tak będzie. Poradzimy sobie bez tego. Czas wyciągnąć wnioski" 
– mówił we wrześniu premier Mateusz Morawiecki.
Bruksela odmawia ustępstw. Oprócz zamrożonych funduszy na odzyskiwanie strat spowodowanych COVID UE grozi teraz Polsce 
wycofaniem 75 mld euro z funduszy rozwojowych, jeśli kraj nie wprowadzi zmian w sądownictwie. "Z powodu braku 
zgodności Komisja nie może pozytywnie rozpatrzyć wniosków o płatność" – powiedział bez ogródek w październiku 
rzecznik UE Stefan de Keersmaecker.
W obliczu wyborów w Polsce jesienią 2023 r. oraz perspektywy ogólnoeuropejskiego kryzysu gospodarczego, rzeczywistość, 
w której w niedalekiej przyszłości zabraknie prawie 110 mld euro, zaczyna być jednak odczuwalna. Zarówno agencje 
ratingowe Moodys, jak i Fitch ostrzegły, że wstrzymanie budżetu UE może mieć tragiczne konsekwencje gospodarcze i 
rynkowe: Polska pilnie potrzebuje unijnych pieniędzy, aby poradzić sobie z gwałtownie rosnącą inflacją i rosnącym 
zagrożeniem spowolnienia gospodarczego.
W sytuacji, gdy dostawy rosyjskiego gazu spadły już do zera, większość polskiej debaty politycznej skupia się na 
groźbie zbliżającego się niedoboru węgla, który może sprawić, że niektóre gospodarstwa domowe nie będą w stanie 
ogrzać się tej zimy. Przed wojną w Ukrainie Polska importowała około 7 mln ton węgla rocznie z Rosji. Ta liczba 
obecnie również spadła do zera. Tymczasem przykładowo władze miejskie Łodzi mogą stracić nawet 8 mld zł (1,71 mld euro)
, z czego polski rząd może zwrócić jedynie marne 160 mln zł (34,13 mln euro). Taka dysproporcja to "katastrofa 
inwestycyjna" dla samorządów w całej Polsce powiedział lokalny polityk Sebastian Bulak. "Będziemy musieli zgasić 
światło".
"""

In [77]:
extract_keyphrases(Warszawa_odmawia_przyjęcia_prymatu_UE_long)

[('działania rewizyjne prowadzone', 9.0), ('ostatnich miesiącach utarło', 9.0), ('zdecydowanie największy wzrost', 9.0), ('głównymi siłami ue', 9.0), ('uznaniu ogromnej roli', 9.0), ('wojennej mobilizacji europy', 9.0), ('otrzymanie pierwszych płatności', 9.0), ('znak ocieplenia więzi', 9.0), ('miesiąc miodowy skończył', 9.0), ('radykalnym szymonem szynkowskim', 9.0), ('czas wyciągnąć wnioski', 9.0), ('bruksela odmawia ustępstw', 9.0), ('pozytywnie rozpatrzyć wniosków', 9.0), ('niedalekiej przyszłości zabraknie', 9.0), ('wstrzymanie budżetu ue', 9.0), ('gwałtownie rosnącą inflacją', 9.0), ('niektóre gospodarstwa domowe', 9.0), ('oprócz zamrożonych funduszy', 8.5), ('jaką polska odegrała', 8.0), ('ukrainie polska importowała', 8.0), ('czego polski rząd', 8.0), ('funduszy rozwojowych', 4.5), ('kontaktach polski', 4.0), ('jarosław kaczyński', 4.0), ('przeciwdziałania autorytaryzmowi', 4.0), ('środek ciężkości', 4.0), ('szybko stała', 4.0), ('świecącym przykładem', 4.0), ('europejskiej sol